In [ ]:
!pip install MLflow
import mlflow
mlflow.autolog()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
# @title Importation données
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
ct = '/content/drive/MyDrive/QueryResults (1).csv'
df = pd.read_csv(ct)


Mounted at /content/drive


In [ ]:
# @title Filtre des tags

df['Tags'] = df['Tags'].str.replace('><', ',').str.replace('<', '').str.replace('>', '')

from collections import Counter
# Fréquence des mots dans 'Tags'
all_tags = df['Tags'].str.split(',').explode().tolist()
tag_counts = Counter(all_tags)

# Trouver les 50 mots les plus fréquents
top_50_tags = [tag for tag, _ in tag_counts.most_common(50)]

# Filtrer les listes de mots pour ne contenir que les 50 mots les plus fréquents
def filter_tags(tags):
    return [tag for tag in tags if tag in top_50_tags]

df['Tags'] = df['Tags'].str.split(',').apply(filter_tags)

# Filtrer les lignes où 'Tags' est vide après le filtrage
df = df[df['Tags'].apply(len) > 0]

df['nb_tags'] = df['Tags'].apply(lambda x: len(x))
df = df[df['nb_tags']==3]

<ipython-input-3-c9400e09b6b9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nb_tags'] = df['Tags'].apply(lambda x: len(x))


In [ ]:
# @title Nettoyage du texte
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Nettoyage
def tokenizer_fct(sentence):
    """Division de mots en texte + suppression de certains caractères"""
    sentence_clean = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    word_tokens = word_tokenize(sentence_clean)
    return word_tokens

stop_w = list(set(stopwords.words('english'))) + ['[', ']', ',', '.', ':', '?', '(', ')','<','>','~']

def stop_word_filter_fct(list_words):
    """Suppression de mots sans information+ ponctuations"""
    filtered_w = [w for w in list_words if not w in stop_w]
    filtered_w2 = [w for w in filtered_w if len(w) > 2]
    return filtered_w2

def lower_start_fct(list_words):
    """Conversion en lettres minuscules et suppression de préfixes indésirables"""
    lw = [w.lower() for w in list_words if (not w.startswith("@")) and (not w.startswith("#")) and (not w.startswith("http"))]
    return lw

def lemma_fct(list_words):
    """lemmatisation"""
    lemmatizer = WordNetLemmatizer()
    lem_w = [lemmatizer.lemmatize(w) for w in list_words]
    return lem_w

def transform_bow_fct(desc_text):
    """fonction de transformation"""
    word_tokens = tokenizer_fct(desc_text)
    sw = stop_word_filter_fct(word_tokens)
    lw = lower_start_fct(sw)
    transf_desc_text = ' '.join(lw)
    return transf_desc_text

# Prétraitement
df['Cleaned_Body'] = df['Body'].apply(transform_bow_fct)


2023/11/24 21:17:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# @title Encoding TF - IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_array = tfidf_vectorizer.fit_transform(df['Cleaned_Body']).toarray()

# Liste des termes correspondant aux colonnes de la matrice
feature_names = tfidf_vectorizer.get_feature_names_out()
# Scores de chaque mot
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

# Concaténez les DataFrames sans utiliser de sample
df.reset_index(drop=True, inplace=True)
df2 = pd.concat([df, tfidf_df], axis=1)

In [ ]:
# @title Modèle supervisé
import random as rd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
rd.seed(0)

def modelisation(enc,name):
    # Convertir les étiquettes ('Tags') en un format binaire
    mlb = MultiLabelBinarizer()
    #binary_labels = mlb.fit_transform(df['Tags']).tolist()
    # Diviser les données pour l'entraînement et le test
    X_train, X_test, y_train, y_test = train_test_split(enc, df['Tags'], test_size=0.2, random_state=42)
    y_train = [y[rd.randint(0, 2)] for y in y_train]
    y_test = [y[rd.randint(0, 2)] for y in y_test]

    # Initialisation et entrainement du modèle
    with mlflow.start_run(run_name= 'supervised_'+name) as run:
        naive_bayes = MultinomialNB()
        naive_bayes.fit(X_train, y_train)
        mlflow.sklearn.log_model(
            sk_model=naive_bayes,
            artifact_path="model",
            registered_model_name="supervised",
        )

    # Prédictions
    predicted_tags = naive_bayes.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_tags)

    print("La précision du modèle supervisé avec ",name," est de ",round(accuracy,2)*100," % ")

modelisation(tfidf_df,'tf_idf')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2023/11/24 21:18:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'supervised'.
Created version '1' of model 'supervised'.


La précision du modèle supervisé avec  tf_idf  est de  20.0  % 


In [ ]:
# @title Extraction de caractéristiques avec Word2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import numpy as np

sentences = [text.split() for text in df['Cleaned_Body']]

# Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Dictionnaire de mots vers vecteurs
word_vectors = word2vec_model.wv

# Obtention du vecteur d'un document en moyennant les vecteurs de ses mots
def document_vector(word_vectors, doc):
    # Supprimer les mots absents du vocabulaire du modèle
    doc = [word for word in doc if word in word_vectors.key_to_index]
    if len(doc) > 0:
        return np.mean(word_vectors[doc], axis=0)
    else:
        return np.zeros(word_vectors.vector_size)

word2vec_features = np.array([document_vector(word_vectors, doc) for doc in sentences])

word2vec_df = pd.DataFrame(word2vec_features, columns=[f'w2v_{i}' for i in range(word2vec_features.shape[1])])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
word2vec_df = scaler.fit_transform(word2vec_df)
modelisation(word2vec_df,'Word2Vec')

2023/11/24 21:19:18 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


La précision du modèle supervisé avec  Word2Vec  est de  12.0  % 


Registered model 'supervised' already exists. Creating a new version of this model...
Created version '2' of model 'supervised'.


In [ ]:
# @title Extraction de caractéristiques avec USE

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

# Modèle Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Obtention du vecteur d'un document
def get_use_embedding(text):
    # Convertir le texte en embeddings
    embeddings = embed([text])
    return np.array(embeddings).flatten()


use_features = np.array([get_use_embedding(text) for text in df['Cleaned_Body']])

use_df = pd.DataFrame(use_features, columns=[f'use_{i}' for i in range(use_features.shape[1])])
use_df = scaler.fit_transform(use_df)

modelisation(word2vec_df,'USE')

2023/11/24 21:19:35 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2023/11/24 21:20:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


La précision du modèle supervisé avec  USE  est de  12.0  % 


Registered model 'supervised' already exists. Creating a new version of this model...
Created version '3' of model 'supervised'.


In [ ]:
!pip install transformers

In [10]:
# @title Extraction de caractéristiques avec BERT

from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np

# Modèle BERT et le tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Obtention du vecteur d'un document
def get_bert_embedding(text):
    # Tokenizer le texte et obtenir les embeddings
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Prendre la sortie correspondant au [CLS] token
    embeddings = outputs.last_hidden_state[:, 0, :].detach().numpy()
    return np.array(embeddings).flatten()


bert_features = np.array([get_bert_embedding(text) for text in df['Cleaned_Body']])

bert_df = pd.DataFrame(bert_features, columns=[f'bert_{i}' for i in range(bert_features.shape[1])])
bert_df = scaler.fit_transform(bert_df)

modelisation(bert_df,'BERT')

2023/11/24 21:20:59 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of transformers. If you encounter errors during autologging, try upgrading / downgrading transformers to a supported version, or try upgrading MLflow.
2023/11/24 21:21:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for transformers.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

2023/11/24 23:05:57 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'list' object has no attribute 'flatten'
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


La précision du modèle supervisé avec  BERT  est de  13.0  % 


Registered model 'supervised' already exists. Creating a new version of this model...
Created version '4' of model 'supervised'.
